# Constructing a property dataset for Australian investors

onthehouse.com.au provides suburb level property data which provides insights for all suburbs in Australia. I'm going to try and construct a dataset from this.

## Exposing the API

The API is quite easy to understanding.

Here is an example of the request for the insights of Bonnyrigg (a NSW state): `https://www.onthehouse.com.au/odin/api/marketstats/markets/trends/BONNYRIGG/NSW/2177?propertyType=House&timePeriod=5`

All we need is to insert `Bonnyrigg` (with `+` if there's a space in the suburb name) and its postcode `2177`.

Both elements can be scrapped in full from these links (for each suburb): `https://www.onthehouse.com.au/suburb-research/nsw` 

I will first scrape all suburb names and postcodes, and then plug them into the API and construct a database from that.

## Gathering suburb names and postcodes

From `https://www.onthehouse.com.au/suburb-research/nsw`, there is a long list of these suburbs and their postcodes.

I used this script on each page (there were 10 pages for NSW) to extract the JSON data:

In [ ]:
// const listItems = Array.from(document.querySelectorAll('ul.d-sm-flex.flex-wrap.StateSuburbList__ssSuburblist--NizcF > li'));
// const extractedData = listItems.map(li => {
//   const anchorTag = li.querySelector('a');
//   if (anchorTag) {
//     return {
//       href: anchorTag.href,
//       text: anchorTag.textContent.trim()
//     };
//   }
//   return null;
// }).filter(item => item !== null);

// console.log(extractedData);

The JSON file `NSW_suburbs.json` is the result of this exercise.

Now to create an array in python:

In [ ]:
import json

with open('NSW_suburbs.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

suburb_postcodes = [
    {
        'suburb': name.strip().upper().replace(' ', '+'),
        'postcode': code.strip()
    }
    for entry in data
    for name, code in (entry['text'].split(',', 1),)
]

print(suburb_postcodes[:10])


## Scraping the analytics data

Before scraping, I want to make sure that my IP address wotn get banned.

First, I'll check if the site has a robots.txt file:

In [ ]:
from urllib.robotparser import RobotFileParser

rp = RobotFileParser()
rp.set_url("https://www.onthehouse.com.au/robots.txt")
rp.read()

url = "https://www.onthehouse.com.au/odin/api/marketstats/markets/trends/AARONS+PASS/NSW/2850?propertyType=House&timePeriod=5"
print(rp.can_fetch("*", url))  # True means robots.txt permits it

In [ ]:
import requests
import time

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
resp = requests.get(url, headers=headers)
print(resp.status_code, resp.headers.get("Retry-After"))

time.sleep(1)  # don’t hammer the server

All looks ok to scrape, so lets do it (TAKES ABOUT AN HOUR AND A HALF):

In [ ]:
#!/usr/bin/env python3
import json
import time
import requests

BASE_URL = "https://www.onthehouse.com.au/odin/api/marketstats/markets/trends"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json",
}

all_trends = []

for entry in suburb_postcodes:
    suburb   = entry['suburb']
    postcode = entry['postcode']
    url = f"{BASE_URL}/{suburb}/NSW/{postcode}?propertyType=House&timePeriod=5"
    
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
        record = resp.json()
    except requests.exceptions.RequestException as e:
        print(f"[ERROR]   {suburb}, {postcode} → {e}")
        continue
    except ValueError:
        print(f"[ERROR]   {suburb}, {postcode} → invalid JSON")
        continue

    # wrap the JSON under a 'data' key along with suburb name
    all_trends.append({
        'suburb': suburb,
        'data':   record
    })
    print(f"[SUCCESS] {suburb}, {postcode} → record added")

    time.sleep(1)  # polite pause

print(f"Fetched {len(all_trends)} records.")

with open("test_market_trends_NSW.json", "w", encoding="utf-8") as f:
    json.dump(all_trends, f, ensure_ascii=False, indent=2)


In [ ]:
import json
import pandas as pd

payload = json.load(open("test_market_trends_NSW.json"))
records = []

for loc in payload:
    # no longer using loc['suburb']
    try:
        series_list = loc['data']['seriesResponseList']
    except KeyError:
        suburb = loc.get('suburb', '<UNKNOWN>')
        print(f"  ⚠️  Missing seriesResponseList for suburb {suburb!r}; skipping.")
        continue

    for metric in series_list:
        # build a combined Locality–Postcode identifier
        locality = metric.get('localityName', '<NO_NAME>')
        postcode = metric.get('postcodeName', '<NO_CODE>')
        loc_id = f"{locality}, {postcode}"

        # sanitize metric name for use as a column later
        m = (metric['metricType']
             .replace(' ', '_')
             .replace('(', '')
             .replace(')', '')
        )

        for point in metric.get('seriesDataList', []):
            d = pd.to_datetime(point['dateTime']).strftime('%Y%m%d')
            records.append({
                'Locality': loc_id,
                'Metric':   m,
                'Date':     d,
                'Value':    point['value']
            })

flat_df = pd.DataFrame.from_records(records)

In [ ]:
# — optional: inspect any true duplicates —
dups = flat_df[flat_df.duplicated(['Locality','Metric','Date'], keep=False)]
if not dups.empty:
    print("Found duplicate rows; here they are:")
    print(dups)

# drop exact duplicates (if that’s safe for your use case)
flat_df = flat_df.drop_duplicates(['Locality','Metric','Date'])

# now pivot using pivot_table+aggfunc to guard against any remaining dupes
wide = flat_df.pivot_table(
    index='Locality',
    columns=['Metric','Date'],
    values='Value',
    aggfunc='first'     # or 'mean' / sum, as makes sense
)

# flatten the MultiIndex columns
wide.columns = [f"{metric}_{date}" for metric, date in wide.columns]
wide = wide.reset_index()

In [ ]:
df = pd.DataFrame(wide)

In [ ]:
# save to CSV
df.to_csv("market_trends_NSW.csv", index=False, encoding='utf-8-sig')